In [16]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json

In [90]:
client_id=os.getenv("CLIENT_ID")
client_secret=os.getenv("CLIENT_SECRET")

In [93]:
client_id

'f9b296ad9d4c43b8a71ce417a5e34301'

In [183]:
def get_token():
    auth_string=client_id+':'+client_secret
    auth_bytes=auth_string.encode('utf-8')
    auth_base64=str(base64.b64encode(auth_bytes),'utf-8')
    
    url='https://accounts.spotify.com/api/token'
    
    headers={
        "Authorization": "Basic "+auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data={"grant_type": "client_credentials"}
    result=post(url,headers=headers,data=data)
    json_result=json.loads(result.content)
    token=json_result["access_token"]
    
    return token

def get_header(token):
    return {"Authorization":"Bearer "+token}

In [149]:
def search_artist(token,artist_name):
    url='https://api.spotify.com/v1/search'
    headers=get_header(token)
    query=f'?q={artist_name}&type=artist&limit=1'
    
    query_url=url+query
    
    result=get(query_url,headers=headers)
    json_result=json.loads(result.content)['artists']['items']
    if len(json_result)==0:
        return None
    
    return json_result[0]

search_artist(token,"Zach Bryan")['id']

'40ZNYROS4zLfyyBSs2PGe2'

In [227]:
def search_track(token,track_name,artist_name):
    url='https://api.spotify.com/v1/search'
    headers=get_header(token)
    query=f'?q={track_name}%{artist_name}&type=track&limit=1'
    
    query_url=url+query
    
    result=get(query_url,headers=headers)
    json_result=json.loads(result.content)['tracks']['items']
    if len(json_result)==0:
        return None
    
    return json_result[0]

track=search_track(token,"Non-Stop","Lin-Manuel")
print(track['name'])
print([i['name'] for i in track["artists"]])

Non-Stop
['Leslie Odom Jr.', 'Lin-Manuel Miranda', 'Renée Elise Goldsberry', 'Phillipa Soo', 'Christopher Jackson', 'Original Broadway Cast of Hamilton']


In [230]:
def list_to_names(artists):
    if len(artists)==1:
        return artists[0]
    output=''
    for artist in artists:
        if artists.index(artist)!=len(artists)-1:
            output+=f'{artist}, '
        else:
            output+=f'{artist}'
    return output

list_to_names([i['name'] for i in search_track(token,"Nonstop","Lin-Manuel")["artists"]])

'Leslie Odom Jr., Lin-Manuel Miranda, Renée Elise Goldsberry, Phillipa Soo, Christopher Jackson, Original Broadway Cast of Hamilton'

In [193]:
def songs_by_artist(token,artist_id):
    url=f'https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US'
    headers=get_header(token)
    result=get(url,headers=headers)
    json_result=json.loads(result.content)['tracks']
    return json_result

token=get_token()
artist_id=search_artist(token,"coldplah")['id']
songs=songs_by_artist(token,artist_id)
for idx,song in enumerate(songs):
    print(f"{idx+1}. {song['name']}")

1. Yellow
2. Viva La Vida
3. Something Just Like This
4. Sparks
5. Christmas Lights
6. A Sky Full of Stars
7. The Scientist
8. Fix You
9. Paradise
10. Hymn for the Weekend


In [129]:
songs_by_artist(token,artist_id)#['items']

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/40ZNYROS4zLfyyBSs2PGe2'},
     'href': 'https://api.spotify.com/v1/artists/40ZNYROS4zLfyyBSs2PGe2',
     'id': '40ZNYROS4zLfyyBSs2PGe2',
     'name': 'Zach Bryan',
     'type': 'artist',
     'uri': 'spotify:artist:40ZNYROS4zLfyyBSs2PGe2'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/6PbnGueEO6LGodPfvNldYf'},
   'href': 'https://api.spotify.com/v1/albums/6PbnGueEO6LGodPfvNldYf',
   'id': '6PbnGueEO6LGodPfvNldYf',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273e5a25ed08d1e7e0fbb440cef',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02e5a25ed08d1e7e0fbb440cef',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851e5a25ed08d1e7e0fbb440cef',
     'width': 64}],
   'is_playable': True,
   'name': 'Zach Bryan',
   'release_date': '2023-08-25'